In [1]:
from libra.retrievers import VectorRetriever
from libra.vectordb import MilvusStorage
from libra.embeddings import SentenceTransformerEncoder, OpenAIEmbedding

import os
from dotenv import load_dotenv

load_dotenv(override=True)

embedding_instance = SentenceTransformerEncoder()

/home/gcp_dpf/libra-core/.venv/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [32]:
mb_info_storage_instance = MilvusStorage(
    vector_dim=embedding_instance.get_output_dim(),
    url_and_api_key=(os.environ['MILVUS_URI'], "123123"),
    collection_name="MBInfo"
)
mb_info_storage_instance.clear()

In [33]:
mb_info_retriever = VectorRetriever(
    embedding_model=embedding_instance, 
    storage=mb_info_storage_instance
)

### Adding Company Profile

In [34]:
from libra.loaders import read_file
from io import BytesIO
import json
from libra.utils import json_to_well_formatted

company_profiles = []


file_path = os.path.join('..', '..', 'data', 'company_profile.csv')
with open(file_path, "rb") as f:
    file = BytesIO(f.read())
    file.name = "company_profile.csv"
    json_file = read_file(file)
    for doc in json_file.docs:
        doc_content = json.loads(doc['page_content'])    
        company_profiles.append({
            "embed": doc_content['Documents Topic'],
            "content": json_to_well_formatted({
                "Topic": doc_content['Documents Topic'],
                "Content": doc_content['Document Content'],
            })
        })

In [35]:
mb_info_retriever.process(data=company_profiles, batch_size=1)

100%|██████████| 169/169 [00:26<00:00,  6.48it/s]


In [39]:
results = mb_info_retriever.query(query="MB có chi nhánh nào gần Hà Đông không ?", top_k=4)

for entry in results:
    print(entry['text'])

Topic: Trụ sở chính hay hội sở của MB Bank nằm ở đâu ở đâu?
Content: Trụ sở chính của MB Bank hiện đang được đặt tại số 18 Lê Văn Lương, Trung Hòa, Cầu Giấy, Hà Nội. 

Tuyến đường Láng Hạ - Lê Văn Lương còn được ví von như là "Phố Wall thứ 2" của Hà Nội bên cạnh dãy phố phía Nam và phía Đông của hồ Hoàn Kiếm (hồ Gươm)

Nếu như bạn muốn đến trụ sở của MB, mình xin được gửi bạn link Google Map: https://maps.app.goo.gl/nDU4cmmtTU4mDX5TA

Bạn hãy xuống các tầng hầm B1, B2, B3, ... để gửi xe nhé. Vì là ngân hàng quân đội nên sẽ có các chú công an hướng dẫn các gửi xe cho bạn.

Và đừng quên là, nếu như bạn cần phải đi lên các tầng làm việc của MB theo lời mời làm việc thì bạn hãy chuẩn bị thư / email mời cũng như căn cước công dân / giấy tờ tùy thân theo để làm thủ tục ở lễ tân nhé

Topic: Các phòng giao dịch và chi nhánh MB ở Khu vực Hà Nội 1/5
Content: Khu vực Hà Nội,Hà Nội,CN. Bà Triệu,Trụ sở,Hà Nội,Quận Hoàn Kiếm,Số 18 Đường Ngô Quyền - Phường Tràng Tiền - Quận Hoàn Kiếm - Hà Nội
Khu vực

### Adding jobs

In [2]:
job_storage_instance = MilvusStorage(
    vector_dim=embedding_instance.get_output_dim(),
    url_and_api_key=(os.environ['MILVUS_URI'], "123123"),
    collection_name="jobs"
)
job_storage_instance.clear()

In [3]:
job_retriever = VectorRetriever(
    embedding_model=embedding_instance,
    storage=job_storage_instance
)

In [16]:
import re

def parse_salaries(salary_data):
    salaries = ("", "")
    lines = salary_data.split('\n')
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        
        # Check for "a - b VND" format
        match = re.match(r'(\d{1,3}(?:,\d{3})*) - (\d{1,3}(?:,\d{3})*)\s*VNĐ', line)
        if match:
            min_salary = match.group(1).replace(',', '')
            max_salary = match.group(2).replace(',', '')
        else:
            # For other formats, set min and max to the same value
            # Remove "VNĐ" and any leading/trailing whitespace
            value = line.replace('VNĐ', '').strip()
            
            if value in ['Lương thỏa thuận', 'Cạnh Tranh']:
                min_salary = max_salary = value
            else:
                # Try to extract a number if present
                number_match = re.search(r'\d{1,3}(?:,\d{3})*', value)
                if number_match:
                    salary_value = int(number_match.group().replace(',', ''))
                    min_salary = max_salary = salary_value
                else:
                    min_salary = max_salary = value

        salaries = (min_salary, max_salary)
    
    return salaries

def is_number(s):
    return s.isdigit()

In [18]:
from libra.loaders import read_file
from io import BytesIO
import json
from pprint import pprint

jobs = []

keys_to_extract = ["workplace", "industry", "welfare", "min_salary", "max_salary"]

file_path = os.path.join('..', '..', 'data', 'jobs_data.json')
with open(file_path, "rb") as f:
    file = BytesIO(f.read())
    file.name = "jobs_data.json"
    json_file = read_file(file)
    for doc in json_file.docs:
        doc_content = json.loads(doc['page_content']) 
        a, b = parse_salaries(doc_content['salary'])
        if is_number(a):
            a +=' VNĐ' # type: ignore
            
        if is_number(b):
            b +=' VNĐ' # type: ignore
        
        doc_content['min_salary'] = a
        doc_content['max_salary'] = b
            
        job = {
            "embed": str({key: doc_content[key] for key in keys_to_extract}),
            "content": str(doc_content)
        }
        jobs.append(job)

In [31]:
jobs[-1]['embed']

"{'workplace': 'Hà Nội', 'industry': 'Ngân hàng, Tài chính / Đầu tư', 'welfare': ['Bảo hiểm', 'Du Lịch', 'Thưởng', 'Chăm sóc sức khỏe', 'Đào tạo', 'Tăng lương'], 'min_salary': 'Lương thỏa thuận', 'max_salary': 'Lương thỏa thuận'}"

In [21]:
job_storage_instance.clear()

In [22]:
job_retriever.process(jobs, batch_size=1)

100%|██████████| 365/365 [02:41<00:00,  2.26it/s]


In [30]:
q = {
    'workplace': 'nghệ an', 
    'industry': 'CNTT', 
    # 'welfare': 'Thưởng', 
    'max_salary': '40 triệu'
}
for entry in job_retriever.query(str(q), top_k=4):
    print(entry['metadata'])

{'embed': "{'workplace': 'Hà Nội', 'industry': 'CNTT - Phần mềm, CNTT - Phần cứng / Mạng, Ngân hàng', 'welfare': [], 'min_salary': 'Lương thỏa thuận', 'max_salary': 'Lương thỏa thuận'}"}
{'embed': "{'workplace': 'Hà Nội', 'industry': 'CNTT - Phần mềm, CNTT - Phần cứng / Mạng, Ngân hàng', 'welfare': [], 'min_salary': 'Lương thỏa thuận', 'max_salary': 'Lương thỏa thuận'}"}
{'embed': "{'workplace': 'Hà Nội', 'industry': 'CNTT - Phần mềm, CNTT - Phần cứng / Mạng, Ngân hàng', 'welfare': [], 'min_salary': 'Lương thỏa thuận', 'max_salary': 'Lương thỏa thuận'}"}
{'embed': "{'workplace': 'Hà Nội', 'industry': 'CNTT - Phần mềm, CNTT - Phần cứng / Mạng, Ngân hàng', 'welfare': [], 'min_salary': 'Lương thỏa thuận', 'max_salary': 'Lương thỏa thuận'}"}
